In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

import os
import numpy as np
import scipy as sc
import matplotlib.pyplot as plt

import pyfrac
from pyfrac.utils.sofiutils import *
from pyfrac.utils.modellingutils import *
from pyfrac.hydrafrac.acquisition import recgeom,recgeom_rockblock

In [2]:
data_dir = '/home/birniece/Documents/data/ANPERC_Lab/LargeRockBlock/Sandstone'

# Rock Properties - subsurface models

In [3]:
modname = 'homo40cubed'

In [4]:
dimlims = 0.40  # 40cm cubed
dx = dy = dz = 0.01  # 5mm
nx = ny = nz = int(dimlims/dx)
# dimlims = nx*dx

# Subsurface properties
vp = 5260  # m/s
vs = 3260  # m/s
rho = 2590;  # kg/m3


In [5]:
subsurface_properties = [vp, vs, rho]
vp_mod, vs_mod, rho_mod = make_homo_model(dimlims,
                                            dx,
                                            subsurface_properties,
                                            pad=False,
                                            npad=0,
                                            freesurface=False
                                            )

vp_mod.astype(np.float32).tofile(os.path.join(data_dir,'%s.vp'%modname))#,format='<f')
vs_mod.astype(np.float32).tofile(os.path.join(data_dir,'%s.vs'%modname))  #,format='<f')
rho_mod.astype(np.float32).tofile(os.path.join(data_dir,'%s.rho'%modname))  #,format='<f')

# Acquisition Geometry - Receivers & Source

## RECEIVERS

In [6]:
# RECEIVERS
nrperface = 38

# don't need to change below
nxyz = [nx, ny, nz]
dxyz = [dx, dy, dz]
gridded_recs, nr, rec_string = recgeom(nxyz, dxyz, nrperfc_xyz=[nrperface], recbuf_gps=1)
# gridded_recs, nr = recgeom_rockblock(nxyz, dxyz, dr_xyz=dx, recbuf_gps=1)
print (nr, gridded_recs.shape)
print(gridded_recs)

np.savetxt(os.path.join(data_dir,'RecArray_fullcoverage.dat'),
           gridded_recs.T.astype(np.float32), 
           fmt='%.2f')

8664 (3, 8664)
[[0.01 0.01 0.01 ... 0.37 0.38 0.39]
 [0.02 0.02 0.02 ... 0.39 0.39 0.39]
 [0.02 0.03 0.04 ... 0.4  0.4  0.4 ]]


## SOURCES 

In [7]:
fc_kHz = 25  # kHz
fs_kHz = 100

fc = fc_kHz * 1000
fs = fs_kHz * 1000
rec_dt = 1/fs

amp = 1.  # amplitude

print(f"Event Central Frequency: {fc},\nSampling Frequency: {fs},\nReceiver dt: {rec_dt}")

Event Central Frequency: 25000,
Sampling Frequency: 100000,
Receiver dt: 1e-05


### Source file 1 - Explosive, heading out one direction but not overlapping

In [8]:
frac_dur = 0.5 # second
# let it just propagate straight x-line
grid_sx_locs = np.round(np.arange(0.01,0.2000001,dx)[::-1], decimals=2)
ignition_times = np.linspace((1/fs),(1/fs) + frac_dur,len(grid_sx_locs))
print(ignition_times)

[1.00000000e-05 2.63257895e-02 5.26415789e-02 7.89573684e-02
 1.05273158e-01 1.31588947e-01 1.57904737e-01 1.84220526e-01
 2.10536316e-01 2.36852105e-01 2.63167895e-01 2.89483684e-01
 3.15799474e-01 3.42115263e-01 3.68431053e-01 3.94746842e-01
 4.21062632e-01 4.47378421e-01 4.73694211e-01 5.00010000e-01]


In [9]:
# MAKE STRINGS PER SHOT LOC
sz = 0.2
sy = 0.2
string_list = []
for t0,sx in zip(ignition_times, grid_sx_locs):
    string_list.append([str(i) for i in [sx, sz, sy, t0, fc, amp]])

# Save to file
src_filename = 'SourceArray_explosiveXfrac.dat'
# print(src_filename)
src_file = os.path.join(data_dir,src_filename)  
# print(src_file)  
text_file = open(src_file, "wt")
for string_line in string_list:
    text_file.writelines('\t'.join(string_line)+'\n')
text_file.close()

In [10]:
# MAKE STRINGS PER SHOT LOC
sz = 0.2
sy = 0.2
td = 1/fs  # ignition time

string_list = []
for sx in grid_sx_locs:
    string_list.append([str(i) for i in [sx, sz, sy, td, fc, amp]])

# Save to file
src_filename = 'SourceArray_explosiveXfrac_singleshots.dat'
# print(src_filename)
src_file = os.path.join(data_dir,src_filename)  
# print(src_file)  
text_file = open(src_file, "wt")
for string_line in string_list:
    text_file.writelines('\t'.join(string_line)+'\n')
text_file.close()

### Source file 2 - independent shots, full x-y coverage

In [11]:
# Want to place directly on grid, dx=4
grid_sx_locs = np.arange(0.1,0.4,dx)
grid_sy_locs = np.arange(0.1,0.4,dy)

sx, sy, sz = np.meshgrid(grid_sx_locs,
                         grid_sy_locs,
                         0.2) 

srclocs = np.vstack((sx.flatten(), sy.flatten(), sz.flatten()))# y is y, I fix the SOFI ordering later
srclocs.shape

src_xyz_list = []
for srcloc in srclocs.T:
    src_xyz_list.append(srcloc)

# MAKE STRINGS PER SHOT LOC
td = 1/fs  # ignition time

src_xyz_list=src_xyz_list[::2]
string_list = []
for src in src_xyz_list:
    string_list.append([str(i) for i in [src[0], src[2], src[1], td, fc, amp]])

# Save to file
src_filename = 'SourceArray_multishotFullXY.dat'
# print(src_filename)
src_file = os.path.join(data_dir,src_filename)  
# print(src_file)  
text_file = open(src_file, "wt")
for string_line in string_list:
    text_file.writelines('\t'.join(string_line)+'\n')
text_file.close()